In [1]:
import tensorflow as tf
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import io,os,sys,types
from IPython import get_ipython #for import notebook
from nbformat import read #for import notebook
from IPython.core.interactiveshell import InteractiveShell #for import notebook

In [3]:
def find_notebook(fullname, path=None): #for import notebook
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [4]:
class NotebookLoader(object): #for import notebook
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [5]:
sys.meta_path.append(NotebookFinder())

In [6]:
import myprepdata

importing Jupyter notebook from myprepdata.ipynb


In [7]:
def my_func(a):
    return tf.log(tf.abs(a)+1)*(a/tf.abs(a))

In [8]:
class naive_encoder(object): #python 2 3 compatibility
    def __init__(self, layers, directions  , maxv, hiddensizes , bach_s):
    
        self.layer = layers
        self.direction = directions
        self.maxseq = maxv
        self.hiddensize = hiddensizes
        self.bach_size = bach_s
        with tf.variable_scope('encoder_input') as scope:
            self.input = tf.placeholder(dtype="float32",shape=[self.bach_size, self.maxseq, self.hiddensize]) 
            #input_hist = tf.summary.histogram("inputv_hist",self.input)
            
        with tf.variable_scope('encoder_gru') as scope:
            self.gru = tf.nn.rnn_cell.GRUCell(num_units = self.hiddensize, activation=tf.nn.relu)
            self.en_output, self.hidden = tf.nn.dynamic_rnn(self.gru, inputs = self.input, dtype=tf.float32)       
            #en_hist = tf.summary.histogram("en_hist",self.en_output)
            #hidden_hist = tf.summary.histogram("hidden_hist",self.hidden)

    
    def hidden_init(self,sess):
        return sess.run(tf.global_variables_initializer())
        
 
    
    def getoutput(self):
        return self.en_output
    
    def getstate(self):
        return self.hidden
'''
    def train(self, sess, input_v):
        summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter("./testgru")
        writer.add_graph(sess.graph)
        #print("hidden = ",sess.run([self.hidden], feed_dict={self.input:input_v}))
        #print("en = ",sess.run([self.en_output], feed_dict={self.input:input_v}))
        s = sess.run(summary,feed_dict={self.input:input_v})
        writer.add_summary(s,0)
        
        return sess.run([self.hidden, self.en_output], feed_dict={self.input:input_v})
'''

'\n    def train(self, sess, input_v):\n        summary = tf.summary.merge_all()\n        writer = tf.summary.FileWriter("./testgru")\n        writer.add_graph(sess.graph)\n        #print("hidden = ",sess.run([self.hidden], feed_dict={self.input:input_v}))\n        #print("en = ",sess.run([self.en_output], feed_dict={self.input:input_v}))\n        s = sess.run(summary,feed_dict={self.input:input_v})\n        writer.add_summary(s,0)\n        \n        return sess.run([self.hidden, self.en_output], feed_dict={self.input:input_v})\n'

In [9]:

class attention_decoder(object):
    def __init__(self, layers, directions, hiddensize, input_length, 
                   teacher_forcing, encoder): # encoder_out, state will be feed
        
        bach_size = 100
        layers = 1
        directions = 1
        hiddensize = 100
        self.hiddensize = hiddensize
        self.def_output_length = 2
        self.enc = encoder
        input_length = 484
        
        with tf.variable_scope('encoder_output') as scope:
            self.input_v = encoder.getoutput()
            
        with tf.variable_scope('realsent') as scope:
            self.realsent = tf.placeholder(dtype='float32')
            
        with tf.variable_scope('decoder_output_segment') as scope:
            #self.output = tf.Variable(tf.zeros([1, self.hiddensize]),dtype='float32') #my output segment
            self.output = tf.placeholder(shape=(1,self.hiddensize),dtype='float32')
            
        with tf.variable_scope('decoder_res_seq') as scope:
            self.output_length = tf.placeholder(dtype='float32')
            self.out_length = tf.placeholder(dtype='float32')
             
        with tf.variable_scope('encoder_state') as scope:
            self.grustate = encoder.getstate()
        teacher_forcing = False
        
        with tf.variable_scope('decoder_var_W') as scope:
            self.W1 = tf.Variable(tf.random_normal([self.hiddensize*2, input_length], stddev=0.1), dtype='float32')
            self.W2 = tf.Variable(tf.random_normal([self.hiddensize*2, self.hiddensize], stddev=0.1), dtype='float32')
            self.W3 = tf.Variable(tf.random_normal([self.hiddensize, self.hiddensize], stddev=0.1),dtype='float32')
            
        with tf.variable_scope('decoder_var_b') as scope:
            self.b1 = tf.Variable(tf.random_normal([1, input_length], stddev=0.1), dtype='float32')
            self.b2 = tf.Variable(tf.random_normal([1, self.hiddensize], stddev=0.1), dtype='float32')
            #self.b3 = tf.Variable(tf.random_normal([3, self.hiddensize], stddev=0.1), dtype='float32')##
            
            self.W1 = tf.clip_by_value(self.W1,-3,3)
            self.b1 = tf.clip_by_value(self.b1,-3,3)
            self.W2 = tf.clip_by_value(self.W2,-10,10)
            self.b2 = tf.clip_by_value(self.b2,-10,10)
            
        with tf.variable_scope("mygru") as scope:
            self.gru = tf.nn.rnn_cell.GRUCell(num_units = 100, activation=tf.nn.relu)
        
        with tf.variable_scope('encoder_concat') as scope:
            encoder_output_state = tf.concat([self.grustate, self.output],1) # 1*100 + 1*100 = 1*200
        
        with tf.variable_scope('makesent') as scope:
            self.last_status = tf.scan(self._nt_atten, self.output_length , initializer = encoder_output_state) #my full output
        
        with tf.variable_scope('sent_postprocess') as scope:
            pre_sentence = tf.squeeze(self.last_status) # 1 3 1 200 -> 3 200
            _ , self.sentence = tf.split(pre_sentence, num_or_size_splits=2, axis = 1) # 3 200 -> ignore(3,100) acc(3,100)
            
            self.sentence = tf.matmul(self.sentence, self.W3)
            #tf.log(tf.abs(new_output)+1)*(new_output/tf.abs(new_output))
        
        
        
        with tf.variable_scope('calcost') as scope:
            self.precost = (self.realsent-self.sentence) * (self.realsent - self.sentence)
            self.cost = tf.reduce_mean(self.precost)
            
        with tf.variable_scope('adamtrain') as scope:
            self.adamtrain = tf.train.AdamOptimizer(learning_rate=0.01)#.minimize(self.cost) #1
            self.grad = self.adamtrain.compute_gradients(self.cost)
            
            self.capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in self.grad]
            self.train_op = self.adamtrain.apply_gradients(self.capped_gvs)
            
            self.adam2train = tf.train.AdamOptimizer(learning_rate=0.05).minimize(self.cost)
            
        with tf.variable_scope('gradtrain') as scope:
            self.gradtrain = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(self.cost) #2
            
        with tf.variable_scope('gradtrain') as scope:
            self.gradtrain2 = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(self.cost) #3
            
    def _nt_atten(self, grustate_output ,ignore): #not teacher forcing

        with tf.variable_scope("split") as scope:
            pre_new_grustate, pre_new_output = tf.split(grustate_output, num_or_size_splits=2, axis=1)
            _ = ignore
            
        with tf.variable_scope("atten") as scope:
            attn = tf.concat([pre_new_grustate, pre_new_output],1)#concat and linear
            
            attn = tf.add(tf.matmul(attn , self.W1),self.b1) # needs W initialize
            attn = tf.log(tf.abs(attn)+1)*(attn/tf.abs(attn)) #new activation
            #attn = tf.tanh(attn)
            #attn = tf.nn.relu(attn)
            attn = tf.nn.softmax(attn)## tanh

        with tf.variable_scope("bmm") as scope: #make 1 wordlike 1 484 * 484 100 == 1 100
            bmm = tf.matmul( attn,tf.squeeze(self.input_v)) 

        with tf.variable_scope("comb") as scope:
            attn_com = tf.concat([bmm, self.output],1 ) # 1 100 + 1 100 = 1 200
            attn_com = tf.add(tf.matmul(attn_com,self.W2),self.b2) # 1 200 * 200 100 = 1 100
            attn_com = tf.log(tf.abs(attn_com)+1)*(attn_com/tf.abs(attn_com))
            #attn_com = tf.tanh(attn_com)
            #attn_com = tf.nn.relu(attn_com)
            pre_new_output = attn_com
            
        with tf.variable_scope("grucall") as scope:
            new_grustate, new_output =self.gru.call(pre_new_grustate, pre_new_output )

            #word = myprepdata.findwordsfromvec(output.eval())
            #print(word)
            #np_output = myprepdata.findvecfromwords(word)
            #output = tf.Variable(np_output, dtype="float32")
            
            
        with tf.variable_scope("concat") as scope:
            grustate_output = tf.concat([new_grustate, new_output],1) # 1 100 + 1 100 = 1 200
            #grustate_output = tf.reshape(grustate_output, [1,200], name='grustate_output')
            
        return grustate_output

  

    def calcost(self,sess,realsente,output_length,line,out_l): #reduce demention
        return sess.run([self.cost],feed_dict = {self.realsent: realsente, self.output_length:output_length,
                                                      self.enc.input:line, self.out_length:out_l, 
                                                 self.output:np.zeros(shape = (1,100))})
    
    def adamtraining(self,sess,realsente,output_length,line,out_l):
        
        #writer = tf.summary.FileWriter("./testgru")
        #writer.add_graph(sess.graph)
        
        return sess.run([self.train_op],feed_dict = {self.realsent: realsente, self.output_length:output_length,
                                                      self.enc.input:line, self.out_length:out_l,
                                                     self.output:np.zeros(shape = (1,100))})
    def adam2training(self,sess,realsente,output_length,line,out_l):
        
        #writer = tf.summary.FileWriter("./testgru")
        #writer.add_graph(sess.graph)
        
        return sess.run([self.adam2train],feed_dict = {self.realsent: realsente, self.output_length:output_length,
                                                      self.enc.input:line, self.out_length:out_l,
                                                      self.output:np.zeros(shape = (1,100))})
    
    
    def gradtraining(self,sess,realsente,output_length,line,out_l):
        
        #writer = tf.summary.FileWriter("./testgru")
        #writer.add_graph(sess.graph)
        
        return sess.run([self.gradtrain],feed_dict = {self.realsent: realsente, self.output_length:output_length,
                                                      self.enc.input:line, self.out_length:out_l,
                                                     self.output:np.zeros(shape = (1,100))})
    def gradtraining2(self,sess,realsente,output_length,line,out_l):
        
        #writer = tf.summary.FileWriter("./testgru")
        #writer.add_graph(sess.graph)
        
        
        return sess.run([self.gradtrain2],feed_dict = {self.realsent: realsente, self.output_length:output_length,
                                                      self.enc.input:line, self.out_length:out_l,
                                                      self.output:np.zeros(shape = (1,100))})
    def hidden_init(self,sess):
        return sess.run(tf.global_variables_initializer())


In [10]:
dataset_loader = myprepdata.prep()

[  8.89466166e-01   2.06377721e+00   6.17529333e-01   3.18206835e+00
   7.67453134e-01  -1.99592626e+00  -1.48355320e-01  -1.96766481e-01
  -3.16980266e-04   2.14467272e-01  -4.14004624e-01   3.57001126e-01
  -1.75457168e+00  -1.39167953e+00   2.63751793e+00   1.19258475e+00
   6.61663294e-01  -3.63710284e-01   1.10500908e+00  -1.90649366e+00
  -1.27631116e+00   7.06363976e-01   2.16604018e+00   7.20149457e-01
   1.19366109e+00   3.15650672e-01  -3.05436397e+00   7.40569711e-01
   8.19141865e-01   1.18011284e+00   4.69861925e-01   8.27771947e-02
   1.12338221e+00   2.47764182e+00   3.43266904e-01  -2.68325806e+00
  -1.40979004e+00  -1.32848775e+00   7.03199029e-01   9.64148343e-02
   3.22712660e-01  -2.13381246e-01  -1.01692128e+00   1.25775146e+00
   1.02071427e-01  -1.11394691e+00  -2.41053867e+00  -1.19636559e+00
   4.32757884e-01   1.38155115e+00   1.13593802e-01  -1.06976998e+00
   1.57687080e+00   1.43941748e+00   2.68036485e-01   1.28006709e+00
   1.21146309e+00   6.14654839e-01

In [11]:
newline = dataset_loader.prepare_line()[0]
newline

'Res@@ um@@ ption of the session'

In [12]:
deline = dataset_loader.de_prepare_line()[0]
deline

'Wiederaufnahme der Sitzungsperiode'

In [13]:
def make_length_np(length):
    i = 0
    leng = np.array([])
    
    while(i < length):
        leng = np.append(leng,[1])
        i = i + 1
        
    return leng


In [14]:
def make_pad_line(dataset_loader,input_or_output,linenum):
    if input_or_output == 0:
        newline = dataset_loader.prepare_line()[linenum]
        padding_line = np.full((1, 484, 100), 0, dtype="float32")
        i = 0
        while(i < len(newline.split(' '))):
            padding_line[0][i] = dataset_loader.get_wvmodel().wv[newline.split(' ')[i]]
            i = i + 1
    else:
        newline = dataset_loader.de_prepare_line()[linenum]
        padding_line = np.full((1, len(newline.split(' ')), 100), 0, dtype="float32")
        i = 0
        while(i < len(newline.split(' '))):
            padding_line[0][i] = dataset_loader.de_get_wvmodel().wv[newline.split(' ')[i]]
            i = i + 1
            
    return newline, padding_line

In [15]:
newline = dataset_loader.de_prepare_line()[1660]
newline

'Abstimmungen'

In [16]:
len(newline.split(' '))

1

In [17]:
#dataset_loader.de_get_wvmodel().wv['failures']

In [18]:
# = 0
#hile(i < 33):
#    print(i)
#    dataset_loader.de_get_wvmodel().wv[newline.split(' ')[i]]
#    i = i + 1


In [21]:
with tf.Graph().as_default(): #set default graph
    
    encoder1 = naive_encoder(layers=1, directions =1, maxv=484, hiddensizes=100, bach_s=1)
    decoder1 = attention_decoder(layers = 1, directions = 1, hiddensize = 100, input_length = 484, 
                       teacher_forcing = False, encoder = encoder1)


    with tf.Session() as sess:
            
        
        encoder1.hidden_init(sess)
        decoder1.hidden_init(sess)
        linenum = 0
        batch_num = 0
    
        while(batch_num < 300):
            print("##########_new_batch",batch_num)
            
            step = 0
            
            while(step < 30): #iteration for one batch
                print("newstep:",step)
                
                linenum = batch_num*100

                while(linenum < 100 + batch_num*100): #batch size = 100
                    
                    try:
                        newline, padding_line = make_pad_line(dataset_loader,0,linenum)
                        deline, de_padding_line = make_pad_line(dataset_loader,1,linenum)
                    except:
                        print("w2v error")
                        linenum = linenum + 1
                        continue
                    inputdic = {'sess':sess,
                            'realsente':de_padding_line[0], 
                            'output_length': make_length_np(len(deline.split(' '))),
                               'line':padding_line, 'out_l':len(deline.split(' '))}
                    
                    if len(newline.split(' ')) == 1:
                        print("en_length 1")
                        linenum = linenum + 1
                        continue
                    
                    if len(deline.split(' ')) == 1:
                        print("de_length 1")
                        linenum = linenum + 1
                        continue
                    
                    if len(deline.split(' ')) > 10:
                        linenum = linenum + 1
                        continue

                    #if(linenum == 0):
                    #    p = 0
                    #    while(p < 10):
                    #        decoder1.adamtraining(**inputdic)
                    #        p = p + 1
                            
                    decoder1.adamtraining(**inputdic)
                    if(step%10 == 9):
                        print("cost = ########",decoder1.calcost(**inputdic))


                    linenum = linenum + 1

                    ''' --> print value
                    #print("sent############",sess.run([decoder1.sentence],feed_dict={
                    #        decoder1.realsent:de_padding_line[0], 
                    #        decoder1.output_length: make_length_np(len(deline.split(' '))),
                    #           decoder1.enc.input:padding_line, decoder1.out_length:len(deline.split(' '))}))
                    #print("grust############",sess.run([decoder1.grustate,decoder1.input_v],feed_dict={
                    #        decoder1.realsent:de_padding_line[0], 
                    #        decoder1.output_length: make_length_np(len(deline.split(' '))),
                    #           decoder1.enc.input:padding_line, decoder1.out_length:len(deline.split(' ')),
                    #            decoder1.output:np.zeros(shape = (1,100))}))
                    '''

                    ''' --> print trainable value
                    #variables_names =[v.name for v in tf.trainable_variables()]
                    #values = sess.run(variables_names)
                    #for k,v in zip(variables_names, values):
                    #    print(k, v)
                    '''
                    #k = np.reshape(k,(1,-1))
                    
                step = step + 1
            
            batch_num = batch_num + 1
        
            
            
        saver = tf.train.Saver()
        ckpt_path = saver.save(sess,"saved/train1")
        print("save:",ckpt_path)
        
        

##########_new_batch 0
newstep: 0
en_length 1
newstep: 1
en_length 1
newstep: 2
en_length 1
newstep: 3
en_length 1
newstep: 4
en_length 1
newstep: 5
en_length 1
newstep: 6
en_length 1
newstep: 7
en_length 1
newstep: 8
en_length 1
newstep: 9
cost = ######## [2.9505129]
cost = ######## [5.3552547]
cost = ######## [5.2862978]
cost = ######## [4.9732962]
cost = ######## [5.5447116]
cost = ######## [5.7055235]
cost = ######## [5.3204756]
cost = ######## [3.4908237]
en_length 1
cost = ######## [4.3716269]
cost = ######## [4.4639649]
cost = ######## [3.8924348]
cost = ######## [5.6999841]
cost = ######## [4.364234]
cost = ######## [11.371503]
cost = ######## [4.7253542]
cost = ######## [4.5933261]
cost = ######## [5.1891041]
newstep: 10
en_length 1
newstep: 11
en_length 1
newstep: 12
en_length 1
newstep: 13
en_length 1
newstep: 14
en_length 1
newstep: 15
en_length 1
newstep: 16
en_length 1
newstep: 17
en_length 1
newstep: 18
en_length 1
newstep: 19
cost = ######## [2.9617648]
cost = ######## 

In [16]:
with tf.Graph().as_default(): #set default graph
    
    encoder1 = naive_encoder(layers=1, directions =1, maxv=484, hiddensizes=100, bach_s=1)
    decoder1 = attention_decoder(layers = 1, directions = 1, hiddensize = 100, input_length = 484, 
                       teacher_forcing = False, encoder = encoder1)
    
    with tf.Session() as sess:
        encoder1.hidden_init(sess)
        decoder1.hidden_init(sess)
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())
        load_path = saver.restore(sess,"saved/train1")
        linenum = 0
        batch_num = 0
        while(batch_num < 100):
            print("##########_new_batch",batch_num)

            step = 0

            while(step < 30): #iteration for one batch
                print("newstep:",step)

                linenum = batch_num*300

                while(linenum < 300 + batch_num*300): #batch size = 100

                    try:
                        newline, padding_line = make_pad_line(dataset_loader,0,linenum)
                        deline, de_padding_line = make_pad_line(dataset_loader,1,linenum)
                    except:
                        print("w2v error")
                        linenum = linenum + 1
                        continue
                    inputdic = {'sess':sess,
                            'realsente':de_padding_line[0], 
                            'output_length': make_length_np(len(deline.split(' '))),
                               'line':padding_line, 'out_l':len(deline.split(' '))}

                    if len(newline.split(' ')) == 1:
                        print("en_length 1")
                        linenum = linenum + 1
                        continue

                    if len(deline.split(' ')) == 1:
                        print("de_length 1")
                        linenum = linenum + 1
                        continue

                    if len(deline.split(' ')) > 20:
                        linenum = linenum + 1
                        continue

                    #if(linenum == 0):
                    #    p = 0
                    #    while(p < 10):
                    #        decoder1.adamtraining(**inputdic)
                    #        p = p + 1

                    decoder1.adamtraining(**inputdic)
                    if(step%10 == 9):
                        print("cost = ########",decoder1.calcost(**inputdic))


                    linenum = linenum + 1

                    ''' --> print value
                    #print("sent############",sess.run([decoder1.sentence],feed_dict={
                    #        decoder1.realsent:de_padding_line[0], 
                    #        decoder1.output_length: make_length_np(len(deline.split(' '))),
                    #           decoder1.enc.input:padding_line, decoder1.out_length:len(deline.split(' '))}))
                    #print("grust############",sess.run([decoder1.grustate,decoder1.input_v],feed_dict={
                    #        decoder1.realsent:de_padding_line[0], 
                    #        decoder1.output_length: make_length_np(len(deline.split(' '))),
                    #           decoder1.enc.input:padding_line, decoder1.out_length:len(deline.split(' ')),
                    #            decoder1.output:np.zeros(shape = (1,100))}))
                    '''

                    ''' --> print trainable value
                    #variables_names =[v.name for v in tf.trainable_variables()]
                    #values = sess.run(variables_names)
                    #for k,v in zip(variables_names, values):
                    #    print(k, v)
                    '''
                    #k = np.reshape(k,(1,-1))

                step = step + 1

            batch_num = batch_num + 1


        sec_ckpt_path = saver.save(sess,"saved/train2")
        print("save:",sec_ckpt_path)

INFO:tensorflow:Restoring parameters from saved/train1
##########_new_batch 0
newstep: 0
en_length 1
newstep: 1
en_length 1
newstep: 2
en_length 1
newstep: 3
en_length 1
newstep: 4
en_length 1
newstep: 5
en_length 1
newstep: 6
en_length 1
newstep: 7
en_length 1
newstep: 8
en_length 1
newstep: 9
cost = ######## [2.7772992]
cost = ######## [6.5440512]
cost = ######## [5.5680933]
cost = ######## [5.4405346]
cost = ######## [3.6572232]
cost = ######## [4.3517208]
cost = ######## [5.5601501]
cost = ######## [3.5309794]
cost = ######## [6.5774136]
cost = ######## [5.0591216]
cost = ######## [4.0342169]
cost = ######## [4.203043]
cost = ######## [5.0772781]
cost = ######## [4.2933669]
cost = ######## [5.1086164]
cost = ######## [5.2443066]
cost = ######## [5.7820864]
cost = ######## [5.6975341]
cost = ######## [5.2829866]
cost = ######## [5.3479171]
cost = ######## [4.6862135]
cost = ######## [4.6996365]
cost = ######## [4.0123062]
cost = ######## [3.9994125]
cost = ######## [6.272779]
cost =

In [ ]:
with tf.Graph().as_default(): #set default graph
    
    encoder1 = naive_encoder(layers=1, directions =1, maxv=484, hiddensizes=100, bach_s=1)
    decoder1 = attention_decoder(layers = 1, directions = 1, hiddensize = 100, input_length = 484, 
                       teacher_forcing = False, encoder = encoder1)
    
    with tf.Session() as sess:
        encoder1.hidden_init(sess)
        decoder1.hidden_init(sess)
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())
        load_path = saver.restore(sess,"saved/train2")
        linenum = 0
        batch_num = 0
        while(batch_num < 100):
            print("##########_new_batch",batch_num)

            step = 0

            while(step < 30): #iteration for one batch
                print("newstep:",step)

                linenum = batch_num*300

                while(linenum < 300 + batch_num*300): #batch size = 100

                    try:
                        newline, padding_line = make_pad_line(dataset_loader,0,linenum)
                        deline, de_padding_line = make_pad_line(dataset_loader,1,linenum)
                    except:
                        print("w2v error")
                        linenum = linenum + 1
                        continue
                    inputdic = {'sess':sess,
                            'realsente':de_padding_line[0], 
                            'output_length': make_length_np(len(deline.split(' '))),
                               'line':padding_line, 'out_l':len(deline.split(' '))}

                    if len(newline.split(' ')) == 1:
                        print("en_length 1")
                        linenum = linenum + 1
                        continue

                    if len(deline.split(' ')) == 1:
                        print("de_length 1")
                        linenum = linenum + 1
                        continue

                    if len(deline.split(' ')) > 20:
                        linenum = linenum + 1
                        continue

                    #if(linenum == 0):
                    #    p = 0
                    #    while(p < 10):
                    #        decoder1.adamtraining(**inputdic)
                    #        p = p + 1

                    decoder1.adamtraining(**inputdic)
                    if(step%10 == 9):
                        print("cost = ########",decoder1.calcost(**inputdic))


                    linenum = linenum + 1

                    ''' --> print value
                    #print("sent############",sess.run([decoder1.sentence],feed_dict={
                    #        decoder1.realsent:de_padding_line[0], 
                    #        decoder1.output_length: make_length_np(len(deline.split(' '))),
                    #           decoder1.enc.input:padding_line, decoder1.out_length:len(deline.split(' '))}))
                    #print("grust############",sess.run([decoder1.grustate,decoder1.input_v],feed_dict={
                    #        decoder1.realsent:de_padding_line[0], 
                    #        decoder1.output_length: make_length_np(len(deline.split(' '))),
                    #           decoder1.enc.input:padding_line, decoder1.out_length:len(deline.split(' ')),
                    #            decoder1.output:np.zeros(shape = (1,100))}))
                    '''

                    ''' --> print trainable value
                    #variables_names =[v.name for v in tf.trainable_variables()]
                    #values = sess.run(variables_names)
                    #for k,v in zip(variables_names, values):
                    #    print(k, v)
                    '''
                    #k = np.reshape(k,(1,-1))

                step = step + 1

            batch_num = batch_num + 1


        sec_ckpt_path = saver.save(sess,"saved/train3")
        print("save:",sec_ckpt_path)

In [7]:
import tensorflow as tf
import numpy as np
a = tf.constant([-1,2,3,4],dtype="float32")
w = tf.Variable(tf.random_normal(shape=(4,4)),dtype="float32")
p = tf.clip_by_value(w, 0.1, 9)
b = tf.constant([2,2,2,2],dtype="float32")
out = tf.nn.tanh(w*a-b)
cost = (out)*(out)
cost = tf.reduce_mean(cost)
r = tf.constant(0.1,dtype="float32")
opt = tf.train.AdamOptimizer(learning_rate= r ).minimize(cost)

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    print(sess.run(w))
    print(sess.run(cost))
    print(sess.run(opt))
    print(sess.run(w))
    print(sess.run(cost))
    print(sess.run(opt))
    print(sess.run(w))
    print(sess.run(cost))
    print(sess.run(opt))
    print(sess.run(w))
    print(sess.run(cost))
    print(sess.run(opt))
    print(sess.run(w))
    print(sess.run(cost))
    print(sess.run(opt))
    print(sess.run(w))
    print(sess.run(cost))
    print(sess.run(opt))
    saver = tf.train.Saver()
    my_path = saver.save(sess,"savetest/train1")
    print("save:",my_path)

[[ 0.3175092   1.01478803  0.2834807  -0.45139995]
 [ 0.02222689  0.09357494  1.34971428  1.12167847]
 [-0.51542735  0.66359574 -2.0459938  -0.60766172]
 [ 2.27868843  0.58530182 -0.45353267  1.02654302]]
0.807905
None
[[ 0.21751598  0.9147923   0.3834804  -0.35143197]
 [-0.07776923  0.19357362  1.24971569  1.02168083]
 [-0.61542583  0.7635954  -2.01621389 -0.50777322]
 [ 2.17901683  0.68530148 -0.35355026  0.92654413]]
0.762537
None
[[ 0.11747762  0.9761529   0.48321539 -0.25608087]
 [-0.17782331  0.29300386  1.15174448  0.92596561]
 [-0.71552789  0.86330479 -1.97872937 -0.41246623]
 [ 2.07919979  0.78539991 -0.25605148  0.83040601]]
0.703767
None
[[ 0.01735708  1.0393008   0.58333731 -0.16557826]
 [-0.2779848   0.39179221  1.05588913  0.83434522]
 [-0.81581855  0.96030736 -1.93352652 -0.32203287]
 [ 1.97927868  0.88499445 -0.16161746  0.73746228]]
0.640984
None
[[-0.08289497  1.06260228  0.68091357 -0.07892537]
 [-0.37831688  0.4901472   0.96129912  0.74516445]
 [-0.91639555  1.04687

In [2]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    chk = saver.restore(sess,"./savetest/train1")
    print(sess.run(w))
    print(sess.run(cost))
    print(sess.run(opt))
    print(sess.run(w))
    print(sess.run(cost))
    print(sess.run(opt))
    print(sess.run(w))
    print(sess.run(cost))
    print(sess.run(opt))
    print(sess.run(w))
    print(sess.run(cost))
    print(sess.run(opt))
    print(sess.run(w))
    print(sess.run(cost))
    print(sess.run(opt))
    print(sess.run(w))
    print(sess.run(cost))
    print(sess.run(opt))
    
    

ValueError: No variables to save

In [ ]:
a = tf.Variable(10,dtype="float32")
b = tf.log(a)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(b))

In [ ]:
a = tf.placeholder(dtype='int32')
b = tf.zeros(shape=(a,100))

In [ ]:
clip_by_value(
    t,
    clip_value_min,
    clip_value_max,
    name=None
)
